# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
nav = webdriver.Chrome()

In [3]:
def n2brlg (url):
    nav.get(url)
    n = nav.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
    return n

In [4]:
#get values
#usd
usd = n2brlg("https://www.google.com/search?q=usd+brl&sxsrf=AJOqlzXicZ9jR5A-XqQ3Ck0CbM2XzWk1pA%3A1673627713265&source=hp&ei=QYjBY5XnDfXW5OUPi8GS-Ak&iflsig=AK50M_UAAAAAY8GWUUyV-fyoSZbVGSy350yFBCPjiZS_&ved=0ahUKEwjVi9WK_cT8AhV1K7kGHYugBJ8Q4dUDCAk&uact=5&oq=usd+brl&gs_lcp=Cgdnd3Mtd2l6EAMyCQgjECcQRhCCAjIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECCMQJzoRCC4QgAQQsQMQgwEQxwEQ0QM6DgguEIAEELEDEMcBENEDOgsILhCABBCxAxCDAToLCAAQgAQQsQMQgwE6CAguEIAEELEDOggIABCABBCxAzoICAAQsQMQgwE6CggAEIAEELEDEA06DQgAEIAEELEDEIMBEA06CggAELEDEIMBEAo6BwgAEIAEEA06CAgAEIAEEMsBOgcIABCABBAKUABYmwpgxw1oAHAAeACAAYABiAH2BZIBAzEuNpgBAKABAQ&sclient=gws-wiz")
#eur
eur = n2brlg("https://www.google.com/search?q=eur+brl&ei=vYvBY67JEPOf5OUP18mQiAE&ved=0ahUKEwjum4O0gMX8AhXzD7kGHdckBBEQ4dUDCBA&uact=5&oq=eur+brl&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQAzIPCAAQsQMQgwEQQxBGEIICMgQIABBDMgcIABCABBAKMgcIABCABBAKMgcIABCABBAKMgcIABCABBAKMgcIABCABBAKMgcIABCABBAKMgYIABAHEB4yBwgAEIAEEAo6CggAEEcQ1gQQsAM6BwgAELADEEM6CggAELEDEIMBEEM6CQgAEIAEEA0QCjoMCAAQgAQQDRBGEIICOgcIABCABBANOggIABCABBCxAzoLCC4QgAQQxwEQrwE6CwgAEIAEELEDEIMBSgQIQRgASgQIRhgAUK8LWMsYYOoZaAFwAXgAgAGzAYgB9waSAQMwLjaYAQCgAQHIAQrAAQE&sclient=gws-wiz-serp")
#gold
nav.get("https://www.melhorcambio.com/ouro-hoje")
gold = nav.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
gold = gold.replace(",",".")
nav.quit()

In [5]:
print('usd: ',usd,'\neur: ',eur,'\ngold: ',gold)

usd:  5.0847 
eur:  5.503933515 
gold:  313.67


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [6]:
tab = pd.read_excel("Produtos.xlsx")

#Atualizar dados

In [7]:

tab.loc[tab["Moeda"] == "Dólar","Cotação"] = float(usd)
tab.loc[tab["Moeda"] == "Euro","Cotação"] = float(eur)
tab.loc[tab["Moeda"] == "Ourp","Cotação"] = float(gold)

In [8]:

tab["Preço de Compra"] = tab["Cotação"]*tab["Preço Original"]

In [9]:
tab["Preço de Venda"] = tab["Margem"]*tab["Preço de Compra"]

In [10]:
display(tab)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.084700,5084.649153,1.40,7118.508814
1,Carro Renault,4500.00,Euro,5.503934,24767.700818,2.00,49535.401635
2,Notebook Dell,899.99,Dólar,5.084700,4576.179153,1.70,7779.504560
3,IPhone,799.00,Dólar,5.084700,4062.675300,1.70,6906.548010
4,Carro Fiat,3000.00,Euro,5.503934,16511.800545,1.90,31372.421035
5,Celular Xiaomi,480.48,Dólar,5.084700,2443.096656,2.00,4886.193312
6,Joia 20g,20.00,Ouro,350.000000,7000.000000,1.15,8050.000000


### Agora vamos exportar a nova base de preços atualizada

In [11]:
tab.to_excel("cotacao.xlsx", index=False)